## Dec 2022
## Explore PLR excel report, rank top species to notice

In [1]:
!pip install openpyxl

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
plt.rcParams['figure.dpi'] = 600
sns.set(style="whitegrid")

/Users/huangz36/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
oligo_nt_set = {'a', 'c', 'g', 't', '-', 'A', 'C','G', 'T'}

## For each row [unique infectious disease strain id], calculate features and assign rankings

In [37]:
## Foward/reverse primers
def count_no_mismatches(oligo_string):
    '''
    Calculate no. mismatch and frac of mismatch in the oligo
    '''
    no_match_count = 0
    oligo_string.replace(".", "-")
    for letter in oligo_string:
        if letter != "." or letter != "-":
            no_match_count+=1
    fraction_mismatch = round(no_match_count/len(oligo_string), 2)
    return (no_match_count, fraction_mismatch)



def three_prime_end_mismatch(oligo_string, no_3_prime_nts):
    '''
    Calculate no. of mismatch at 3' end of the oligo
    '''
    non_match_count = 0
    three_prime = oligo_string[-no_3_prime_nts:]
    for letter in three_prime:
        if letter != ".":
            non_match_count+=1
    return non_match_count

def count_primer_relative_freq(oligo_sequence, df, oligo_column_name):
    df_count_agg = df[['id', oligo_column_name]].groupby(oligo_column_name).count().reset_index()
    freq_count = df_count_agg.loc[df_count_agg[oligo_column_name]==oligo_sequence, 'id'].values[0]
    total_ids = df.shape[0]
    freq_as_ratio = round(int(freq_count)/int(total_ids), 2)
    return freq_count, freq_as_ratio

def input_PLR_return_taxonIDs_need_notice(PLR_file, PLR_sheet_name, no_top_taxon_to_show, three_prime_distance):
    '''
    PLR_file: path to the PLR excel sheet
    PLR_sheet_name: the name of the excel tab of "*_report", with oligo related information
    no_top_taxon_to_show: no. of the most worth noticing taxons to show, that may not be amplified by assay
    three_prime_distance: for forward and reverse primers, the n nucleotides to 3' end that needs mismatch check
    '''
    ## Read in dataframe, oligo names and column names
    df = pd.read_excel(PLR_file, sheet_name=PLR_sheet_name)
    oligo_header = df.columns.to_list()
    complete_header = df.iloc[0].to_list()

    ## Merge and organize new header
    new_header = []
    for column_no in range(len(oligo_header)):
        if oligo_header[column_no].startswith("Unnamed"):
            new_col_name = complete_header[column_no]
        elif set(complete_header[column_no]).issubset(oligo_nt_set):
            new_col_name = oligo_header[column_no]
        else:
            new_col_name = str(str(oligo_header[column_no]).split(".")[0]) + ":" + str(complete_header[column_no])
        new_header.append(new_col_name)

    ## Rename df with new_header
    df = df[1:]
    df.columns = new_header

    ## For primer oligo sequence columns ending with (f) and (r), make mismatch related calculations
    ## For primer oligo sequence, also count frequency and relative abundance of the haplotype
    primer_sequence_columns = []
    three_prime_rank_columns = []
    no_mismatch_rank_columns = []
    mismatch_ratio_rank_columns = []
    haplotype_count_columns = []
    for col_names in new_header:
        #print("col_names", col_names)
        if col_names.endswith("(f)") or col_names.endswith("(r)"): # the oligo columns
            three_prime_mismatch_name = str(col_names) + "_3_prime_mismatch"
            mismatch_name = str(col_names) + "_mismatch_count"
            mismatch_ratio_name = str(col_names) + "_mismatch_ratio"
            haplotype_count_name = str(col_names) + "_haplotype_count"
            haplotype_ratio_name = str(col_names) + "_relative_abundance"

            df[three_prime_mismatch_name] = df[col_names].apply(three_prime_end_mismatch, args=(three_prime_distance, ))
            df[[mismatch_name, mismatch_ratio_name]]= df[col_names].apply(count_no_mismatches).tolist()
            df[[haplotype_count_name, haplotype_ratio_name]] = df[col_names].apply(count_primer_relative_freq, args=(df, col_names)).tolist()

            three_prime_rank_columns.append(three_prime_mismatch_name)
            no_mismatch_rank_columns.append(mismatch_name)
            mismatch_ratio_rank_columns.append(mismatch_ratio_name)
            haplotype_count_columns.append(haplotype_ratio_name)
            primer_sequence_columns.append(col_names)
            
            columns_for_strain_ranking = three_prime_rank_columns + \
            no_mismatch_rank_columns + mismatch_ratio_rank_columns + haplotype_count_columns
        ## Pending 
        ## need do develop for "p", once quencher and dye location obtained

    ascending_list = [False] * len(columns_for_strain_ranking)
    df_need_notice = df.sort_values(columns_for_strain_ranking, ascending = ascending_list)

    ## Rank strain IDs
    to_show_columns = ['Isolate_Id', 'Isolate_Name', 'Location', 'Host', 'Collection_Date', 'country']
    to_show_extended = to_show_columns + primer_sequence_columns + haplotype_count_columns \
    + three_prime_rank_columns + mismatch_ratio_rank_columns
    df_need_notice = df.sort_values(columns_for_strain_ranking, \
                                    ascending = ascending_list).head(no_top_taxon_to_show)[to_show_extended]
    
    return df_need_notice


In [34]:
#PLR_file = '/Users/huangz36/Documents/Hackathon_2022/Galaxy2243.PLR.xlsx'
#sheet_name = 'Yers_report'

#PLR_file = '/Users/huangz36/Documents/Hackathon_2022/FluA_hackathon.xlsx'

PLR_file = '/Users/huangz36/Documents/Hackathon_2022/flu_hackathon_1100.xlsx'
sheet_name = 'FluA_report'


df = pd.read_excel(PLR_file, sheet_name=sheet_name)

In [35]:
output_df = input_PLR_return_taxonIDs_need_notice(PLR_file, sheet_name, 10, 4)

In [36]:
output_df

,id,Isolate_Id,Isolate_Name,Location,Host,Collection_Date,country,FluA_assay(f),FluA_assay(r),FluA_assay(f)_relative_abundance,FluA_assay(r)_relative_abundance,FluA_assay(f)_3_prime_mismatch,FluA_assay(r)_3_prime_mismatch,FluA_assay(f)_mismatch_ratio,FluA_assay(r)_mismatch_ratio
640,A/Hawaii/46/2022|EPI_ISL_15928821||State_of_Ha...,EPI_ISL_15928821,A/Hawaii/46/2022,North America / United States / Hawaii,Human,2022-10-19 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
705,A/Hawaii/45/2022|EPI_ISL_15928807||State_of_Ha...,EPI_ISL_15928807,A/Hawaii/45/2022,North America / United States / Hawaii,Human,2022-10-18 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
762,A/Colorado/36/2022|EPI_ISL_15928796||Colorado_...,EPI_ISL_15928796,A/Colorado/36/2022,North America / United States / Colorado,Human,2022-10-16 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
786,A/Hawaii/43/2022|EPI_ISL_15928787||State_of_Ha...,EPI_ISL_15928787,A/Hawaii/43/2022,North America / United States / Hawaii,Human,2022-10-15 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
864,A/Hawaii/42/2022|EPI_ISL_15928830||State_of_Ha...,EPI_ISL_15928830,A/Hawaii/42/2022,North America / United States / Hawaii,Human,2022-10-13 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
895,A/Hawaii/41/2022|EPI_ISL_15928818||State_of_Ha...,EPI_ISL_15928818,A/Hawaii/41/2022,North America / United States / Hawaii,Human,2022-10-12 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
896,A/California/140/2022|EPI_ISL_15928786||Califo...,EPI_ISL_15928786,A/California/140/2022,North America / United States / California,Human,2022-10-12 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
1074,A/Iowa/26/2022|EPI_ISL_15715021||Iowa_State_Hy...,EPI_ISL_15715021,A/Iowa/26/2022,North America / United States / Iowa,Human,2022-10-07 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
1089,A/Iowa/26/2022|EPI_ISL_15928856||Iowa_State_Hy...,EPI_ISL_15928856,A/Iowa/26/2022,North America / United States / Iowa,Human,2022-10-07 00:00:00,United States,...............T.-.......C,..................,0.01,0.71,1,0,1.0,1.0
494,A/Ibra/72211857/2022|EPI_ISL_15926932||Central...,EPI_ISL_15926932,A/Ibra/72211857/2022,Asia / Oman,Human,2022-10-23 00:00:00,Oman,...............T.-......G.,..................,0.00,0.71,1,0,1.0,1.0


In [11]:
output_df = input_PLR_return_taxonIDs_need_notice(PLR_file, sheet_name, 10, 4)
output_df.to_csv("/Users/huangz36/Documents/Hackathon_2022/example_taxon_at_risk.csv", index=False, header=True)
df.to_csv("/Users/huangz36/Documents/Hackathon_2022/processed_PLR_risk.csv", index=False, header=True)